In [1]:
# run this cell to import nltk
import nltk
from os import getcwd

In [2]:


filePath = f"{getcwd()}/../tmp2/"
nltk.data.path.append(filePath)

In [3]:
import numpy as np
import pandas as pd
from nltk.corpus import twitter_samples 

from utils import process_tweet, build_freqs

In [4]:

all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

In [5]:

test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

train_x = train_pos + train_neg 
test_x = test_pos + test_neg

In [6]:
# combine positive and negative labels
train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

In [7]:
# create frequency dictionary
freqs = build_freqs(train_x, train_y)



In [8]:

def sigmoid(z): 

    h = 1/(1+ np.exp(-z))
    
    return h

In [9]:

def gradientDescent(x, y, theta, alpha, num_iters):

    m = x.shape[0]
    
    for i in range(0, num_iters):
        
        
        z = np.dot(x, theta)
        
        
        h = sigmoid(z)
        
        
        J = (-1/m) * (np.dot(y.T, np.log(h)) + np.dot((1-y).T, np.log(1-h)))

       
        theta = theta - (alpha / m ) * np.dot(x.T, (h-y))
        
    
    J = float(J)
    return J, theta

In [10]:

def extract_features(tweet, freqs):

    word_l = process_tweet(tweet)
    
    x = np.zeros((1, 3)) 

    x[0,0] = 1 

    for word in word_l:
        
        # increment the word count for the positive label 1
        x[0,1] += freqs.get((word, 1), 0)
        
        # increment the word count for the negative label 0
        x[0,2] += freqs.get((word, 0.0),0)

    assert(x.shape == (1, 3))
    return x

In [11]:

X = np.zeros((len(train_x), 3))
for i in range(len(train_x)):
    X[i, :]= extract_features(train_x[i], freqs)

Y = train_y

J, theta = gradientDescent(X, Y, np.zeros((3, 1)), 1e-9, 1500)

In [12]:

def predict_tweet(tweet, freqs, theta):

    x = extract_features(tweet,freqs)
    
  
    y_pred = sigmoid(np.dot(x, theta))
        
    return y_pred

In [13]:

for tweet in ['I am happy', 'I am bad', 'this movie should have been great.', 'great', 'great great', 'great great great', 'great great great great']:
    print( '%s -> %f' % (tweet, predict_tweet(tweet, freqs, theta)))

I am happy -> 0.518580
I am bad -> 0.494339
this movie should have been great. -> 0.515331
great -> 0.515464
great great -> 0.530898
great great great -> 0.546273
great great great great -> 0.561561


In [14]:
import MySQLdb

In [15]:
dsn_database = "library"   # e.g. "MySQLdbtest"
dsn_hostname = "localhost"       # e.g.: "mydbinstance.xyz.us-east-1.rds.amazonaws.com"
dsn_port = 3306                        # e.g. 3306 
dsn_uid = "root"             # e.g. "user1"
dsn_pwd = ""            # e.g. "Password123"

In [16]:
conn = MySQLdb.connect(host=dsn_hostname, port=dsn_port, user=dsn_uid, passwd=dsn_pwd, db=dsn_database)

In [17]:
mycursor = conn.cursor()

mycursor.execute("SELECT * FROM comments")

myresult = mycursor.fetchall()

for x in myresult:
    pred = predict_tweet( x[2],freqs, theta)
    label =(int)( pred > 0.5)
    a = x[0]
    b = x[1]
    c = x[2]
    label = str(label)
    # UPDATE `comments` SET `label`='-5' WHERE id = '5'
    # mycursor.execute("UPDATE `comments` SET `label`=label WHERE bid = b and name = a and comment = c")
    # conn.commit()


    sql = "UPDATE comments SET label = %s WHERE comment = %s"
    val = (label, x[2] )
    mycursor.execute(sql, val)
    conn.commit()